In [1]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
import laspy
import shutil

import time
import requests
                    
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [2]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

conf_username = ''
if 'JUPYTERHUB_USER' in os.environ:
    conf_username = os.environ['JUPYTERHUB_USER']
    

conf_remote_path_split = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/split')
conf_remote_path_retiled = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/retiled')
conf_remote_path_norm = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/norm')
conf_remote_path_targets = pathlib.Path('/webdav/vl-laserfarm/' + conf_username +  '/targets')
conf_remote_path_geotiffs = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/geotiffs')
conf_local_tmp = pathlib.Path('/tmp/data')


param_hostname = ''
param_username = ''
param_password = ''
param_minio_server = ''

    

conf_num_files = 1 
conf_visualization_mode = 'webdav'
param_remote_server_type = 'minio'

if param_remote_server_type == 'webdav':
    param_remote_path_root = '/webdav/vl-laserfarm/ahn'
elif param_remote_server_type == 'minio':
    param_bucket_name = 'naa-vre-public'
    param_remote_path_root = 'vl-laserfarm/ahn/'

conf_feature_name = 'perc_95_normalized_height'
conf_validate_precision = '0.001'
conf_tile_mesh_size = '10.'
conf_filter_type= 'select_equal'
conf_attribute = 'raw_classification'
conf_min_x = '-113107.81'
conf_max_x = '398892.19'
conf_min_y = '214783.87'
conf_max_y = '726783.87'
conf_n_tiles_side = '512'
conf_apply_filter_value = '1'
conf_laz_compression_factor = '7'
param_max_filesize = '30'  # desired max file size (in bytes)


conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_username, 'webdav_password': param_password}

In [3]:
#local paths

conf_local_path_split = os.path.join(conf_local_tmp.as_posix(), 'split')
conf_local_path_retiled = os.path.join(conf_local_tmp.as_posix(), 'retiled')
conf_local_path_targets = os.path.join(conf_local_tmp.as_posix(), 'targets')
conf_local_path_geotiff = os.path.join(conf_local_tmp.as_posix(), 'geotiff')

## Fetching Laz Files from remote WebDAV

In [4]:
# S1 Fetch Laz Files

from minio import Minio

laz_files = []
if param_remote_server_type == 'minio':
    minio_client = Minio(param_minio_server, secure=True)
    objects = minio_client.list_objects(param_bucket_name, prefix=param_remote_path_root, recursive=True)
    for obj in objects:
        if obj.object_name.lower().endswith('.laz'):
            laz_files.append(obj.object_name.split('/')[-1])
elif param_remote_server_type == 'webdav':
    print(param_remote_path_root)
    webdva_path = param_remote_path_root
    laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), pathlib.Path(webdva_path).as_posix())
                 if f.lower().endswith('.laz')]
print(laz_files)

['C_01GN1.LAZ', 'C_01GZ2.LAZ', 'C_04FN2.LAZ']


## Splitting big files into smaller files before retiling
This step can be added if the original files are too large for normal VMs to process

In [5]:
# S2 split big files local minio

import shutil
import os
import numpy as np

def save_chunk_to_laz_file(in_filename, 
                           out_filename, 
                           offset, 
                           n_points):
    """Read points from a LAS/LAZ file and write them to a new file."""
    points = np.array([])
    
    with laspy.open(in_filename) as in_file:
        with laspy.open(out_filename, 
                        mode="w", 
                        header=in_file.header) as out_file:
            in_file.seek(offset)
            points = in_file.read_points(n_points)
            out_file.write_points(points)
    return out_filename

def split_strategy(filename, max_filesize):
    """Set up splitting strategy for a LAS/LAZ file."""
    with laspy.open(filename) as f:
        bytes_per_point = (
            f.header.point_format.num_standard_bytes +
            f.header.point_format.num_extra_bytes
        )
        n_points = f.header.point_count
    n_points_target = int(
        max_filesize * int(conf_laz_compression_factor) / bytes_per_point
    )
    stem, ext = os.path.splitext(filename)
    return [
        (filename, f"{stem}-{n}{ext}", offset, n_points_target)
        for n, offset in enumerate(range(0, n_points, n_points_target))
    ]

from webdav3.client import Client

client = Client(conf_wd_opts)
# client.mkdir(conf_remote_path_split.as_posix())
# remote_path_split = str(conf_remote_path_split)
minio_client = Minio(param_minio_server, secure=True)

os.makedirs(conf_local_path_split, exist_ok=True)

for file in laz_files:
    print('Splitting: '+file )
    destination_path = os.path.join(conf_local_tmp,file)
    if param_remote_server_type == 'webdav':
        client.download_sync(remote_path=os.path.join(param_remote_path_root,file), local_path=destination_path)
    elif param_remote_server_type == 'minio':
        minio_client.fget_object(param_bucket_name, os.path.join(param_remote_path_root,file), destination_path)
        print(f"Downloaded: {file} to {destination_path}")
        
    inps = split_strategy(destination_path, int(param_max_filesize)*1048576)
    
    for inp in inps:
        out_filename = save_chunk_to_laz_file(*inp)
        print('out_filename: '+out_filename)
        split_path = os.path.join(conf_local_path_split,os.path.basename(out_filename))
        if os.path.exists(split_path):
            os.remove(split_path)
        shutil.move(out_filename, conf_local_path_split)

S2_done = 'True'

Splitting: C_01GN1.LAZ
Downloaded: C_01GN1.LAZ to /tmp/data/C_01GN1.LAZ
out_filename: /tmp/data/C_01GN1-0.LAZ
out_filename: /tmp/data/C_01GN1-1.LAZ
out_filename: /tmp/data/C_01GN1-2.LAZ
Splitting: C_01GZ2.LAZ
Downloaded: C_01GZ2.LAZ to /tmp/data/C_01GZ2.LAZ
out_filename: /tmp/data/C_01GZ2-0.LAZ
out_filename: /tmp/data/C_01GZ2-1.LAZ
out_filename: /tmp/data/C_01GZ2-2.LAZ
Splitting: C_04FN2.LAZ
Downloaded: C_04FN2.LAZ to /tmp/data/C_04FN2.LAZ
out_filename: /tmp/data/C_04FN2-0.LAZ
out_filename: /tmp/data/C_04FN2-1.LAZ
out_filename: /tmp/data/C_04FN2-2.LAZ


In [6]:
# S21 Fetch Laz Files local
import os
import glob

S2_done 

split_laz_folder = glob.glob(os.path.join(conf_local_path_split, '*.LAZ'))
split_laz_files = []
print("File names ending with .LAZ:")
for file_path in split_laz_folder:
    split_laz_files.append(os.path.basename(file_path))

print(split_laz_files)
S21_done = 'True'

File names ending with .LAZ:
['C_01GN1-0.LAZ', 'C_04FN2-0.LAZ', 'C_01GZ2-1.LAZ', 'C_04FN2-2.LAZ', 'C_01GN1-2.LAZ', 'C_01GZ2-0.LAZ', 'C_01GN1-1.LAZ', 'C_04FN2-1.LAZ', 'C_01GZ2-2.LAZ']


## Retiling of big files into smaller tiles

In [7]:
# S3 Retiling local
split_laz_files
# remote_path_retiled = str(conf_remote_path_retiled)

grid_retile = {
    'min_x': float(conf_min_x),
    'max_x': float(conf_max_x),
    'min_y': float(conf_min_y),
    'max_y': float(conf_max_y),
    'n_tiles_side': int(conf_n_tiles_side)
}

retiling_input = {
    # 'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'setup_local_fs': {
        'input_folder': conf_local_path_split,
        'output_folder': conf_local_path_retiled
    },
    # 'pullremote': conf_remote_path_split.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    # 'pushremote': conf_remote_path_retiled.as_posix(),
    # 'cleanlocalfs': {}
}

for file in split_laz_files:
    clean_file = file.replace('"','').replace('[','').replace(']','')
    print(clean_file)
    # retiler = Retiler(clean_file,label=clean_file).config(retiling_input).setup_webdav_client(conf_wd_opts)
    retiler = Retiler(clean_file,label=clean_file).config(retiling_input)
    retiler_output = retiler.run()

S3_done = 'True'

2024-01-24 17:02:33,424 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/data/split
2024-01-24 17:02:33,426 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/data/retiled
2024-01-24 17:02:33,432 -                        laserfarm.retiler -       INFO - Setting up the target grid
2024-01-24 17:02:33,436 -                        laserfarm.retiler -       INFO - Splitting file /tmp/data/split/C_01GN1-0.LAZ with PDAL ...


C_01GN1-0.LAZ


2024-01-24 17:02:45,831 -                        laserfarm.retiler -       INFO - ... splitting completed.
2024-01-24 17:02:45,835 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2024-01-24 17:02:45,839 -                        laserfarm.retiler -       INFO - ... file C_01GN1-0_1.LAZ to tile_273_391
2024-01-24 17:02:45,841 -                        laserfarm.retiler -       INFO - ... file C_01GN1-0_5.LAZ to tile_276_391
2024-01-24 17:02:45,843 -                        laserfarm.retiler -       INFO - ... file C_01GN1-0_2.LAZ to tile_274_391
2024-01-24 17:02:45,847 -                        laserfarm.retiler -       INFO - ... file C_01GN1-0_3.LAZ to tile_275_391
2024-01-24 17:02:45,850 -                        laserfarm.retiler -       INFO - ... file C_01GN1-0_4.LAZ to tile_275_392
2024-01-24 17:02:45,853 -                        laserfarm.retiler -       INFO - ... redistributing completed.
2024-01-24 17:02:45,854 -                        l

C_04FN2-0.LAZ


2024-01-24 17:02:57,997 -                        laserfarm.retiler -       INFO - ... splitting completed.
2024-01-24 17:02:57,999 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2024-01-24 17:02:58,000 -                        laserfarm.retiler -       INFO - ... file C_04FN2-0_2.LAZ to tile_252_382
2024-01-24 17:02:58,002 -                        laserfarm.retiler -       INFO - ... file C_04FN2-0_5.LAZ to tile_253_380
2024-01-24 17:02:58,004 -                        laserfarm.retiler -       INFO - ... file C_04FN2-0_4.LAZ to tile_252_383
2024-01-24 17:02:58,005 -                        laserfarm.retiler -       INFO - ... file C_04FN2-0_3.LAZ to tile_252_380
2024-01-24 17:02:58,007 -                        laserfarm.retiler -       INFO - ... file C_04FN2-0_6.LAZ to tile_253_381
2024-01-24 17:02:58,009 -                        laserfarm.retiler -       INFO - ... file C_04FN2-0_1.LAZ to tile_252_381
2024-01-24 17:02:58,010 -              

...

2024-01-24 17:03:53,231 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2024-01-24 17:03:53,235 -                        laserfarm.retiler -       INFO - ... file C_01GN1-1_3.LAZ to tile_276_391
2024-01-24 17:03:53,238 -                        laserfarm.retiler -       INFO - ... file C_01GN1-1_1.LAZ to tile_275_391
2024-01-24 17:03:53,241 -                        laserfarm.retiler -       INFO - ... file C_01GN1-1_6.LAZ to tile_277_392
2024-01-24 17:03:53,244 -                        laserfarm.retiler -       INFO - ... file C_01GN1-1_5.LAZ to tile_277_391
2024-01-24 17:03:53,246 -                        laserfarm.retiler -       INFO - ... file C_01GN1-1_2.LAZ to tile_275_392
2024-01-24 17:03:53,249 -                        laserfarm.retiler -       INFO - ... file C_01GN1-1_4.LAZ to tile_276_392
2024-01-24 17:03:53,250 -                        laserfarm.retiler -       INFO - ... redistributing completed.
2024-01-24 17:03:53,252 -         

C_04FN2-1.LAZ


2024-01-24 17:04:05,381 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2024-01-24 17:04:05,384 -                        laserfarm.retiler -       INFO - ... file C_04FN2-1_3.LAZ to tile_252_382
2024-01-24 17:04:05,387 -                        laserfarm.retiler -       INFO - ... file C_04FN2-1_4.LAZ to tile_253_382
2024-01-24 17:04:05,389 -                        laserfarm.retiler -       INFO - ... file C_04FN2-1_1.LAZ to tile_252_381
2024-01-24 17:04:05,392 -                        laserfarm.retiler -       INFO - ... file C_04FN2-1_2.LAZ to tile_253_381
2024-01-24 17:04:05,393 -                        laserfarm.retiler -       INFO - ... redistributing completed.
2024-01-24 17:04:05,395 -                        laserfarm.retiler -       INFO - Validating split ...
2024-01-24 17:04:05,397 -                        laserfarm.retiler -       INFO - ... 7864320 points in parent file
2024-01-24 17:04:05,399 -                        laserfarm.re

C_01GZ2-2.LAZ


2024-01-24 17:04:07,615 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2024-01-24 17:04:07,618 -                        laserfarm.retiler -       INFO - ... file C_01GZ2-2_3.LAZ to tile_283_391
2024-01-24 17:04:07,620 -                        laserfarm.retiler -       INFO - ... file C_01GZ2-2_2.LAZ to tile_279_391
2024-01-24 17:04:07,623 -                        laserfarm.retiler -       INFO - ... file C_01GZ2-2_1.LAZ to tile_279_390
2024-01-24 17:04:07,624 -                        laserfarm.retiler -       INFO - ... redistributing completed.
2024-01-24 17:04:07,626 -                        laserfarm.retiler -       INFO - Validating split ...
2024-01-24 17:04:07,628 -                        laserfarm.retiler -       INFO - ... 1312860 points in parent file
2024-01-24 17:04:07,630 -                        laserfarm.retiler -       INFO - ... 464408 points in C_01GZ2-2_1.LAZ
2024-01-24 17:04:07,633 -                        laserfarm.retile

In [8]:
# S4 Retrive retiled tiles local
# remote_path_retiled

# tiles = [t.strip('/') for t in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_retiled.as_posix())
#          if fnmatch.fnmatch(t, 'tile_*_*/')]
S3_done 

tiles = []
tile_folders = glob.glob(os.path.join(conf_local_path_retiled, 'tile_*_*'))

for folder in tile_folders:
    # Extract only the folder name without the 'conf_local_path_retiled'
    folder_name = os.path.basename(folder)
    tiles.append(folder_name)  # Append only the folder name
print(tiles)

['tile_279_390', 'tile_278_392', 'tile_277_391', 'tile_283_391', 'tile_253_381', 'tile_252_382', 'tile_274_391', 'tile_275_392', 'tile_253_380', 'tile_278_391', 'tile_253_383', 'tile_278_390', 'tile_275_391', 'tile_252_383', 'tile_252_381', 'tile_252_380', 'tile_273_391', 'tile_276_392', 'tile_253_382', 'tile_276_391', 'tile_277_392', 'tile_279_391']


In [9]:
# S5 Feature Extraction local

for t in tiles:
    local_path_targets = os.path.join(conf_local_path_targets, t)
    features = [conf_feature_name]

    tile_mesh_size = float(conf_tile_mesh_size)

    grid_feature = {
        'min_x': float(conf_min_x),
        'max_x': float(conf_max_x),
        'min_y': float(conf_min_y),
        'max_y': float(conf_max_y),
        'n_tiles_side': int(conf_n_tiles_side)
    }

    feature_extraction_input = {
        'setup_local_fs': {
            'input_folder': conf_local_path_retiled,
            'output_folder': local_path_targets
        },
        # 'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
        # 'pullremote': conf_remote_path_retiled.as_posix(),
        'load': {'attributes': [conf_attribute]},
        'normalize': 1,
        'apply_filter': {
            'filter_type': conf_filter_type, 
            'attribute': conf_attribute,
            'value': [int(conf_apply_filter_value)]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
        },
        'generate_targets': {
            'tile_mesh_size' : tile_mesh_size,
            'validate' : True,
            'validate_precision': float(conf_validate_precision),
            **grid_feature
        },
        'extract_features': {
            'feature_names': features,
            'volume_type': 'cell',
            'volume_size': tile_mesh_size
        },
        'export_targets': {
            'attributes': features,
            'multi_band_files': False
        },
        # 'pushremote': conf_remote_path_targets.as_posix(),
    #     'cleanlocalfs': {}
    }
    idx = (t.split('_')[1:])

    # processing = DataProcessing(t, tile_index=idx,label=t).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
    processing = DataProcessing(t, tile_index=idx,label=t).config(feature_extraction_input)
    processing.run()
    target_file = os.path.join(local_path_targets, conf_feature_name,t+'.ply')  
    target_folder = os.path.join(conf_local_path_targets, conf_feature_name)
    os.makedirs(target_folder, exist_ok=True)
    shutil.move(target_file, target_folder)
    
S5_done = 'True'

2024-01-24 17:04:07,722 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/data/retiled
2024-01-24 17:04:07,726 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/data/targets/tile_279_390
2024-01-24 17:04:07,732 -                laserfarm.data_processing -       INFO - Loading point cloud data ...
2024-01-24 17:04:07,735 -                laserfarm.data_processing -       INFO - ... loading /tmp/data/retiled/tile_279_390/C_01GZ2-1_3.LAZ
2024-01-24 17:04:09,152 -                laserfarm.data_processing -       INFO - ... loading /tmp/data/retiled/tile_279_390/C_01GZ2-2_1.LAZ
2024-01-24 17:04:09,529 -                laserfarm.data_processing -       INFO - ... loading completed.
2024-01-24 17:04:09,532 -                laserfarm.data_processing -       INFO - Normalizing point-cloud heights ...
2024-01-24 17:04:10,811 -                                     root -       INFO - Cylinder size in Bytes: 4816211580.418289
2024-01-24 1

In [10]:
# S6 GeoTIFF Export local

S5_done

# setup input dictionary to configure the GeoTIFF export pipeline
geotiff_export_input = {
    'setup_local_fs': {
        'input_folder': conf_local_path_targets,
         'output_folder': conf_local_path_geotiff
        },
    # 'pullremote': conf_remote_path_targets.as_posix(),
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': 'geotiff'},
    'pushremote': conf_remote_path_geotiffs.as_posix(),
    # 'cleanlocalfs': {}   
}

writer = GeotiffWriter(input_dir=conf_feature_name, bands=conf_feature_name, label=conf_feature_name).config(geotiff_export_input).setup_webdav_client(conf_wd_opts)
writer.run()

remote_path_geotiffs = str(conf_remote_path_geotiffs)
S6_done = 'True'

2024-01-24 17:11:44,553 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/data/targets
2024-01-24 17:11:44,558 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/data/geotiff
2024-01-24 17:11:44,563 -                 laserfarm.geotiff_writer -       INFO - 22 PLY files found
2024-01-24 17:11:44,666 -                 laserfarm.geotiff_writer -       INFO - No. of points per file: 10000
2024-01-24 17:11:44,669 -                 laserfarm.geotiff_writer -       INFO - Resolution: (10.0m x 10.0m)
2024-01-24 17:11:44,670 -                 laserfarm.geotiff_writer -       INFO - Splitting data into (1x1) sub-regions
2024-01-24 17:11:44,672 -                 laserfarm.geotiff_writer -       INFO - Processing sub-region GeoTiff no. 0 ...
2024-01-24 17:11:44,674 -                 laserfarm.geotiff_writer -       INFO - ... number of constituent tiles: 22
2024-01-24 17:11:47,858 -                 laserfarm.geotiff_writer -       INFO - 

In [11]:
# param vislulization
S6_done

hostname = param_hostname
username = param_username
password = param_password
remote = str(conf_remote_path_geotiffs)
num = conf_num_files
mode = conf_visualization_mode
output = str(conf_local_tmp)

In [12]:
# print param
print(hostname)
print(username)
print(password)
print(remote)
print(num)
print(mode)
print(output)

https://lifewatch.lab.uvalight.net/
20BNXDdL8mg24OaD
zDoy0hNKkcnsdsQ@OYAVd
/webdav/vl-laserfarm/spiros.koulouzis@lifewatch.eu/geotiffs
1
webdav
/tmp/data


In [14]:
#visualize_rasterio

import os 
import rasterio
from rasterio.plot import show
from matplotlib import pyplot
import numpy as np
from rasterio.plot import show
from rasterio.plot import show_hist

S6_done

geo_tiff = os.path.join(conf_local_path_geotiff, 'geotiff_TILE_000_BAND_perc_95_normalized_height.tif')
src = rasterio.open(geo_tiff)
show(src)
fig, ax = pyplot.subplots(1, figsize=(30, 30))
show((src, 1), interpolation='none', ax=ax)
show((src, 1), contour=True, ax=ax)
pyplot.show()
show_hist(src, bins=50, lw=0.0, stacked=False, alpha=0.3, histtype='stepfilled', title="Histogram")
pyplot.show()

ModuleNotFoundError: No module named 'rasterio'